## Price Optimization
Review codes of example price optimization

-----
**Documentation gurobiML**
https://gurobi-machinelearning.readthedocs.io/en/stable/auto_examples/example4_price_optimization.html#sphx-glr-auto-examples-example4-price-optimization-py

-----
**Link to colab notebooks**

- Price optimization only gurobi: https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/price_optimization/price_optimization.ipynb#scrollTo=4537e1ae

- Price optimization gurobi machine learning (gurobiML): https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/price_optimization/price_optimization_gurobiML.ipynb

- Price optimization gurobiML with licence: https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/price_optimization/price_optimization_gurobiML_wls.ipynb#scrollTo=pzQR12rAr-b_

### 0. Packages

In [ ]:
# ## install gurobi packages

# !pip install gurobipy
# !pip install gurobi-machinelearning
# !pip install gurobipy-pandas

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# sklearn
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

# plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# package gurobi
import gurobipy as gp
from gurobi_ml import add_predictor_constr
import gurobipy_pandas as gppd

### 1. Setear licencia

In [ ]:
# # setear variable de ambiente con la licencia
# import os
# path_licencia_gurobi = "gurobi.lic"
# os.environ ["GRB_LICENSE_FILE"] = path_licencia_gurobi
# print(os.environ["GRB_LICENSE_FILE"])

In [ ]:
# crear modelo con licencia seteada
modelo_prueba = gp.Model('Modelo Prueba')

### 2. Load data

In [ ]:
data_url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/price_optimization/"
avocado = pd.read_csv(
    data_url + "HABdata_2019_2022.csv"
)  # dataset downloaded directly from HAB
avocado_old = pd.read_csv(
    data_url + "kaggledata_till2018.csv"
)  # dataset downloaded from Kaggle

# The date is in different formats in the two data sets and
# need to be converted separately
avocado["date"] = pd.to_datetime(avocado["date"], format="%m/%d/%y %H:%M")
avocado_old["date"] = pd.to_datetime(avocado_old["date"], format="%m/%d/%y")

# Concatenate the two notebooks
avocado = pd.concat([avocado, avocado_old])
avocado

### 3. Prepare dataset
- Agregar índice con el año
- Agregar columna con las temporadas alta. De febrero a Agosto
- Transformar las unidades vendidas a millones de unidades vendidas (units/1000000)
- Elegir solo tipo de palta convencional

In [ ]:
# Add the index for each year from 2015 through 2022
avocado["year"] = pd.DatetimeIndex(avocado["date"]).year
avocado = avocado.sort_values(by="date")

# Define the peak season -> definir meses peak desde febrero hasta agosto
avocado["month"] = pd.DatetimeIndex(avocado["date"]).month
peak_months = range(2, 8)  # <--------- Set the months for the "peak season"


def peak_season(row):
    return 1 if int(row["month"]) in peak_months else 0


avocado["peak"] = avocado.apply(lambda row: peak_season(row), axis=1)


# Scale the number of avocados to millions
avocado["units_sold"] = avocado["units_sold"] / 1000000

# Select only conventional avocados
avocado = avocado[avocado["type"] == "Conventional"]

avocado = avocado[
    ["date", "units_sold", "price", "region", "year", "month", "peak"]
].reset_index(drop=True)

avocado

### 4. EDA
EDA de los datos propio para entenderlos

NULOS

In [ ]:
### NULOS
avocado.isnull().sum()

CANTIDAD DE VALORES ÚNICOS EN VARIABLES DISCRETAS

In [ ]:
list_features_cat = ['region', 'year', 'month', 'peak']
list_features_cat

In [ ]:
for feature_cat in list_features_cat:
    print('FEATURE: ', feature_cat)
    print('number of unique regions: ', avocado[feature_cat].nunique())
    print('unique regions: ', avocado[feature_cat].unique())
    print('\n')

HIST DE TODAS LAS FEATURES - PLOTLY

In [ ]:
avocado.describe()

In [ ]:
def plot_hist(df, features_to_plot):
    """
    Dado un histograma y un listado de features, plotear histograma de cada una de las features
    Args:
        df: dataframe
        features_to_plot: list - features to plot
    """

    # shape sub plots
    number_columns = 1 #fixed
    number_rows = len(features_to_plot)
    
    # create plot
    fig = make_subplots(rows = number_rows, cols = number_columns)
    
    
    # append subplots
    for index_feature in range(number_rows):
        
        # plot
        fig.append_trace(
            #px.histogram(df, x = features_to_plot[index_feature]),
            go.Histogram(x=df[features_to_plot[index_feature]]),
            row=index_feature + 1, 
            col = 1
        )
        
        #Update x-axis title for each subplot
        fig.update_xaxes(title_text=features_to_plot[index_feature], row=index_feature + 1, col=1)
    
    
    fig.update_layout(height=1600, width=600, title_text="Histograms")
    fig.show()

In [ ]:
plot_hist(df = avocado,
         features_to_plot = avocado.columns.tolist()[1:]
         )

### 5. Observe trends in the data
Now, we will infer sales trends in time and seasonality. For simplicity, let’s proceed with data from the United States as a whole.

#### 5.1 Only data form the United States as a whole

In [ ]:
df_Total_US = avocado[avocado["region"] == "Total_US"]
df_Total_US.head()

In [ ]:
plot_hist(df = df_Total_US,
         features_to_plot = df_Total_US.columns.tolist()[1:]
         )

#### 5.2 Sales over the years

In [ ]:
### sales over the years

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
mean = df_Total_US.groupby("year")["units_sold"].mean()
std = df_Total_US.groupby("year")["units_sold"].std()
axes.errorbar(mean.index, mean, xerr=0.5, yerr=2 * std, linestyle="")
axes.set_ylabel("Units Sold (millions)")
axes.set_xlabel("Year")

fig.tight_layout()

In [ ]:
### time series sales over the years

# sort data
df_timeseries = df_Total_US.sort_values(by = ['date'], ascending = True)

# plot timeseries
fig = px.line(df_timeseries, x='date', y="units_sold")
fig.show()

We can see that the sales generally increased over the years, albeit marginally. The dip in 2019 is the effect of the well-documented 2019 avocado shortage that led to avocados nearly doubling in price.

#### 5.3 Seasonality
We will now see the sales trends within a year.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

mean = df_Total_US.groupby("month")["units_sold"].mean()
std = df_Total_US.groupby("month")["units_sold"].std()

axes.errorbar(mean.index, mean, xerr=0.5, yerr=2 * std, linestyle="")
axes.set_ylabel("Units Sold (millions)")
axes.set_xlabel("Month")

fig.tight_layout()

plt.xlabel("Month")
axes.set_xticks(range(1, 13))
plt.ylabel("Units sold (millions)")
plt.show()

We see a Super Bowl peak in February and a Cinco de Mayo peak in May.

#### 5.4 Correlations
Now, we will see how the variables are correlated with each other. The end goal is to predict sales given the price of an avocado, year and seasonality (peak or not).

In [ ]:
# calculate correlations
columns_to_corr = ["units_sold", "price", "year", "peak"]
corr_df = df_Total_US[columns_to_corr].corr()
corr_df = corr_df.round(2)

# plot correlations
fig = px.imshow(corr_df, width=1000, height=500, text_auto=True, labels = dict(x='Correlations of all features'))
fig.update_xaxes(side="top")
fig.show()

In [ ]:
## only correlations of units solds
columns_to_corr = ["units_sold", "price", "year", "peak"]
corr_price = df_Total_US[columns_to_corr].corr()
corr_price = corr_price[["units_sold"]]
corr_price = corr_price.round(2)

## correlations
fig = px.imshow(corr_price, width=1000, height=500, text_auto=True, labels = dict(x='Correlations of "units solds"'))
fig.update_xaxes(side="top")
fig.show()

As expected, the sales quantity has a negative correlation with the price per avocado. The sales quantity has a positive correlation with the year and season being a peak season.

#### 5.5 Regions
Finally, we will see how the sales differ among the different regions. This will determine the number of avocados that we want to supply to each region.

In [ ]:
# create a dataframe with all regions, deleting the total_US
regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains",
]
df = avocado[avocado.region.isin(regions)]

In [ ]:
# plot units sold of each region


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

mean = df.groupby("region")["units_sold"].mean()
std = df.groupby("region")["units_sold"].std()

axes.errorbar(range(len(mean)), mean, xerr=0.5, yerr=2 * std, linestyle="")

fig.tight_layout()

plt.xlabel("Region")
plt.xticks(range(len(mean)), pd.DataFrame(mean)["units_sold"].index, rotation=20)
plt.ylabel("Units sold (millions)")
plt.show()

#### 5.6: Predict the Sales
The trends observed in Part I motivate us to construct a prediction model for sales using the independent variables- price, year, region and seasonality. Henceforth, the sales quantity will be referred to as the predicted demandarn.

In [ ]:
# define X and y
X = df[["region", "price", "year", "peak"]]
y = df[["units_sold"]]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

In [ ]:
X_train.head()

In [ ]:
X_train.info()

Transform the data
- "region" is a categorical feature -> one hot encoding
- standar scaler -> price and year

In [ ]:
feat_transform = make_column_transformer(
    (OneHotEncoder(drop="first"), ["region"]),
    (StandardScaler(), ["price", "year"]),
    ("passthrough", ["peak"]),
    verbose_feature_names_out=False,
    remainder="drop",
)

The regression model is a pipeline consisting of the Column Transformer we just defined and a Linear Regression.

Define it and train it.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

lin_reg = make_pipeline(feat_transform, LinearRegression())
lin_reg.fit(X_train, y_train)

# Get R^2 from test data
y_pred = lin_reg.predict(X_test)
print(f"The R^2 value in the test set is {r2_score(y_test, y_pred)}")

#### inference example

In [ ]:
# one inference example: 
X_inference = X_test.iloc[[0], :]
X_inference

In [ ]:
# predict with the model - DADO UN PRECIO, ESTOY CALCULANDO UNA DEMANDA
demand_predicted = lin_reg.predict(X_inference)
demand_predicted

In [ ]:
# real value
y_test.iloc[[0], :]

#### estoy cambiando el año...., cómo puede generalizar bien este modelo con datos que claramente no conoce a futuro

We can observe a good R2
 value in the test set. We will now train the fit the weights to the full dataset
YO: WHY, POR QUÉ ESTAS HACIENDO ESTO.

In [ ]:
X.head()

In [ ]:
lin_reg.fit(X, y)

y_pred_full = lin_reg.predict(X)
print(f"The R^2 value in the full dataset is {r2_score(y, y_pred_full)}")

## Part III: Optimize for Price and Supply of Avocados

Sabiendo cómo afecta el precio de un aguacate a la demanda, ¿cómo podemos fijar el precio óptimo del aguacate? No queremos fijar el precio demasiado alto, ya que eso podría reducir la demanda y las ventas. Al mismo tiempo, fijar un precio demasiado bajo podría no ser óptimo a la hora de maximizar los ingresos. Entonces, ¿cuál es el punto ideal?

En cuanto a la logística de distribución, queremos asegurarnos de que haya suficientes aguacates en todas las regiones. Podemos abordar estas consideraciones en un modelo de optimización matemática. Un modelo de optimización encuentra la mejor solución de acuerdo con una función objetivo tal que la solución satisfaga un conjunto de restricciones. Aquí, una solución se expresa como un vector de valores reales o valores enteros llamados variables de decisión. Las restricciones son un conjunto de ecuaciones o desigualdades escritas en función de las variables de decisión.

Al comienzo de cada semana, suponga que la cantidad total de productos disponibles es finita. Esta cantidad debe distribuirse a las distintas regiones maximizando al mismo tiempo los ingresos netos. Así que hay dos decisiones clave: el precio de un aguacate en cada región y la cantidad de aguacates asignados a cada región.

Definamos ahora algunos parámetros de entrada y notaciones utilizadas para crear el modelo. el subíndice
  se utilizará para indicar cada región.

Input Parameters
: set of regions,

: predicted demand in region 
 when the avocado per product is 
,

: available avocados to be distributed across the regions,

: cost (
) per wasted avocado,

: cost (
) of transporting a avocado to region 
,

: minimum and maximum price (
) per avocado for reigon 
,

: minimum and maximum number of avocados allocated to region 
,

El siguiente código carga el paquete Python de Gurobi e inicia el modelo de optimización. El valor de
  se establece en
  millones de aguacates, lo que se aproxima al valor medio de oferta semanal según los datos. A modo de ejemplo, consideremos la temporada alta de 2021. El costo de desperdiciar un aguacate se establece en
. El costo de transportar un aguacate oscila entre
  a
  basado en la distancia de cada región a la frontera sur, de donde proviene la mayor parte del suministro de aguacate. Además, podemos fijar el precio de un aguacate para que no exceda
  una pieza.

In [ ]:
# Sets and parameters
B = 30  # total amount ot avocado supply

peak_or_not = 1  # 1 if it is the peak season; 1 if isn't
year = 2022

c_waste = 0.1  # the cost ($) of wasting an avocado


In [ ]:
# the cost of transporting an avocado
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    },
    name="transport_cost",
)

In [ ]:
c_transport = c_transport.loc[regions]
# the cost of transporting an avocado

c_transport

In [ ]:
# Get the lower and upper bounds from the dataset for the price and the number of products to be stocked
a_min = 0  # minimum avocado price in each region
a_max = 2  # maximum avocado price in each region

In [ ]:
df.head() # tengo un dataframe con los valores individuales de unidades vendidaad en cada en cada region

In [ ]:
# genero un dataframe en base a las unidades vendidas mínimas y maximas agrupadas por region
data = pd.concat(
    [
        c_transport,
        df.groupby("region")["units_sold"].min().rename("min_delivery"),
        df.groupby("region")["units_sold"].max().rename("max_delivery"),
    ],
    axis=1,
)

data

Variables de decisión
Definamos ahora las variables de decisión. En nuestro modelo, queremos almacenar el precio y la cantidad de aguacates asignados a cada región. También queremos variables que rastreen cuántos aguacates se prevé que se venderán y cuántos se desperdiciarán. La siguiente notación se utiliza para modelar estas variables de decisión.

  el precio de un aguacate (
) en cada región,

  el número de aguacates suministrados a cada región,

  el número previsto de aguacates vendidos en cada región,

  el número previsto de aguacates desperdiciados en cada región.

  la demanda prevista en cada región.

Todas esas variables se crean usando gurobipy-pandas, con la función gppd.add_vars se les asigna el mismo índice que el m datos.








In [ ]:
import gurobipy as gp
import gurobipy_pandas as gppd

m = gp.Model("Avocado_Price_Allocation")

p = gppd.add_vars(m, data, name="price", lb=a_min, ub=a_max)
x = gppd.add_vars(m, data, name="x", lb="min_delivery", ub="max_delivery")
s = gppd.add_vars(
    m, data, name="s"
)  # predicted amount of sales in each region for the given price).
w = gppd.add_vars(m, data, name="w")  # excess wasteage in each region).
d = gppd.add_vars(
    m, data, lb=-gp.GRB.INFINITY, name="demand"
)  # Add variables for the regression

m.update()

In [ ]:
# Display one of the variables - PRECIO POR CADA REGION (tengo un modelo que predice la cantidad vendida cierto día dado un precio)
p

Establecer el objetivo
A continuación, definiremos la función objetivo: queremos maximizar los ingresos netos. Los ingresos por ventas en cada región se calculan multiplicando el precio de un aguacate en esa región por la cantidad vendida allí. Se incurre en dos tipos de costos: los costos de desperdicio por el exceso de aguacates no vendidos y el costo de transporte de los aguacates a las diferentes regiones.

Los ingresos netos son los ingresos por ventas restados de los costos totales incurridos. Suponemos que los costos de compra son fijos y no están incorporados en este modelo.

Utilizando las variables de decisión definidas, el objetivo se puede escribir de la siguiente ma modelo.

In [ ]:
m.setObjective(
    (p * s).sum() - c_waste * w.sum() - (c_transport * x).sum(), gp.GRB.MAXIMIZE
)

In [ ]:
m

Agregue la restricción de oferta
Introducimos ahora las restricciones. La primera restricción es asegurarse de que el número total de aguacates suministrados sea igual a
, que se puede expresar matemáticamente de la siguiente manera.


El siguiente código agrega esta restricción al modelo.

In [ ]:
m.addConstr(x.sum() == B)
m.update()

Agregar restricciones que definan la cantidad de ventas
A continuación, debemos definir la cantidad de ventas prevista en cada región. Podemos suponer que si ofrecemos más de la demanda prevista, venderemos exactamente la demanda prevista. De lo contrario, vendemos exactamente la cantidad asignada. Por lo tanto, la cantidad de ventas prevista es el mínimo de la cantidad asignada y la demanda prevista, es decir,
. Esta relación se puede modelar mediante las dos restricciones siguientes para cada región
.

 

Estas restricciones asegurarán que la cantidad de ventas
  en la región
  no es mayor que la cantidad asignada ni la demanda prevista. Tenga en cuenta que la función objetivo de maximización intenta maximizar los ingresos de las ventas y, por lo tanto, el optimizador maximizará la cantidad de ventas prevista. Esto supone que el excedente y los costos de transporte son menores que el precio de venta por aguacate. Por lo tanto, estas restricciones junto con el objetivo asegurarán que las ventas sean iguales al mínimo de oferta y demanda prevista.

Agreguemos ahora estas restricciones al modelo.

En este caso, usamos gurobipy-pandas, función add_constrs

In [ ]:
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, x)
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, d)
m.update()

Agregue las restricciones de desperdicio
Finalmente, debemos definir el desperdicio previsto en cada región, dado por la cantidad ofrecida que no se prevé vender. Podemos expresar esto matemáticamente para cada región.
.


Podemos agregar estas restricciones al modelo.

In [ ]:
gppd.add_constrs(m, w, gp.GRB.EQUAL, x - s)
m.update()

Agregue las restricciones para predecir la demanda.
Primero, creamos nuestra entrada para la restricción del predictor.

Las hazañas del marco de datos contendrán características corregidas:

año

pico con el valor de pico_or_not

región que repite los nombres de las regiones.

y la variable precio p.

Está indexado por regiones (predecimos la demanda de forma independiente para cada región).

Muestre el marco de datos para asegurarse de que sea correcto.

In [ ]:
###3 crear dataframe con las variables de decision¿? - 
# tengo que decidir dado la region en la que estoy y el año y el periodo del año, cual va a ser el precio a vender que me maximize mi ganancias
# dado un cierto precio, tengo un modelo que me dice cuánto venderé y con esa info de precio y cantidad puedo conocer mis ganancias
#y luego con el calculo del costo de transorte,)
feats = pd.DataFrame(
    data={
        "region": regions,
        "price": p,
        "year": year,
        "peak": peak_or_not,
    },
    index=regions,
)
feats

In [ ]:
year

In [ ]:
peak_or_not

#### IMPORTANTE los valores de "year" y "peak_or_not" son definidos como parámetros del modelo de optimización (son variables del modelo de predicción, y aquí yo le estoy definiendo)

In [ ]:
regions

#### IMPORTANTE: regions son los valores de todas las regiones posibles. Al final en los datos tengo muchos valores por día de unidades vendidas pero al modelo de optimización le paso una instancia que quiero conocer (una instancia por región). Muchas menos observaciones que el ejemplo de predicción si el estudiante ingresa a la universidad o no dado las notas y las beca ofrecida donde se pasaba un dataset de todos los alumnos y para cada uno había que predecir porque una restricción estaba dada por la cantidad de alumnos

Now, we just need to call add_predictor_constr to insert the constraints linking the features and the demand.

In [ ]:
m

In [ ]:
lin_reg

In [ ]:
feats

In [ ]:
d

In [ ]:
from gurobi_ml import add_predictor_constr

pred_constr = add_predictor_constr(m, lin_reg, feats, d) # d output

pred_constr.print_stats()

Fire Up the Solver
We have added the decision variables, objective function, and the constraints to the model. The model is ready to be solved. Before we do so, we should let the solver know what type of model this is. The default setting assumes that the objective and the constraints are linear functions of the variables.

In our model, the objective is quadratic since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic term is said to be non-convex, and we specify this by setting the value of the Gurobi NonConvex parameter to be 
.

In [ ]:
m.Params.NonConvex = 2
m.optimize()

El solucionador resolvió el problema de optimización en menos de un segundo. Analicemos ahora la solución óptima almacenándola en un marco de datos de Pandas.

In [ ]:
feats

In [ ]:
# ------> OBTENER LOS VALORES DEL MODELO DE OPTIMIZACIÓN
solution = pd.DataFrame(index=regions)

solution["Price"] = p.gppd.X
solution["Allocated"] = x.gppd.X
solution["Sold"] = s.gppd.X
solution["Wasted"] = w.gppd.X
solution["Pred_demand"] = d.gppd.X

opt_revenue = m.ObjVal
print("\n The optimal net revenue: $%f million" % opt_revenue)
solution.round(4)

We can also check the error in the estimate of the Gurobi solution for the regression model.

In [ ]:
pred_constr

In [ ]:
pred_constr.get_error()

In [ ]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)

And the computed features of the regression model in a pandas dataframe.

In [ ]:
pred_constr

In [ ]:
pred_constr.input_values.drop("region", axis=1)

### Conocer los valores de todas las variables de decisión

#### modelo de optimización

In [ ]:
# modelo de optimización
m

#### variables de decisión que forman parte del modelo de machine learning

In [ ]:
# valores del dataframe con las instancias que se pasan al modelo de machine learning para que prediga. Es la matriz X pasada como dataframe
# El dataframe tiene al menos una variable de decisión que es la que se va iterando
pred_constr.input_values

In [ ]:
# predicción del modelo de machine learning que se optiene como solución del modelo de optimización
pred_constr.output_values

#### valores del resto de variables de decisión que no forman parte del modelo de optimización - discovery

In [ ]:
# mostar la variable
p

In [ ]:
# la variable de decisión se puede filtrar por indice numérico
p.iloc[0]

In [ ]:
# la variable de decisión se puede filtrar por índice
p.loc['Great_Lakes']

In [ ]:
# para tener el valor de la variable de decisión se necesita ".x". obtener valores individales
p.loc['Great_Lakes'].x

In [ ]:
# para obtener todos los valores de la variable de decisión
m.getAttr('X', p)

In [ ]:
# # para obtener los valores de la variable de decisión como serie de pandas bien ----> AL FINAL ESTA ES LA MEJOR FORMA PARA OBTENER LOS VALORES
# DE LAS VARIABLES DE DECISIÓN DEL MODELO DE OPTIMIZACIÓN
p.gppd.X

### plot results

Let us now visualize a scatter plot between the price and the number of avocados sold (in millions) for the eight regions.

In [ ]:
fig, ax = plt.subplots(1, 1)

plot_sol = sns.scatterplot(
    data=solution, x="Price", y="Sold", hue=solution.index, s=100
)
plot_waste = sns.scatterplot(
    data=solution,
    x="Price",
    y="Wasted",
    marker="x",
    hue=solution.index,
    s=100,
    legend=False,
)

plot_sol.legend(loc="center left", bbox_to_anchor=(1.25, 0.5), ncol=1)
plot_waste.legend(loc="center left", bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.ylim(0, 5)
plt.xlim(1, 2.2)
ax.set_xlabel("Price per avocado ($)")
ax.set_ylabel("Number of avocados sold (millions)")
plt.show()
print(
    "The circles represent sales quantity and the cross markers represent the wasted quantity."
)